# Recommender Systems

### Learning Objectives:
- [Introduction: Simple Recommender Systems](#Introduction:-Simple-Recommeder-Systems)
- [Offline & Online Evaluation](#Offline-&-Online-Evaluation)
- [Content-based Recommenders](#Content\-based-Recommenders)
- [Collaborative-filtering](#Collaborative\-filtering)
- [Hybrid Systems](#Hybrid-Systems)


# Introduction: Simple Recommender Systems

__Recommender systems__, also referred to as __recommendation systems__, are filtering systems used by many different companies world-wide to be able to recommend products (e.g. movies, clothes, etc) based on user preferences. Unlike __ranking algorithms,__ recommender systems aim to provide recommendations without an explicit input from the user (such as a search query). We obviously cannot recommend _exactly_ what a user wants as we cannot access or process all the information in their brain at the same time. Instead, we can take advantage or users' past ratings, choices and preferences to __predict__ the products the user will most probably like.

How do these systems do what they do? This is question that has become a large topic of research and the current answer is that there are mutliple ways to create recommender systems: each working under different assumptions and algorithms. There are two main broad classifications that we will cover shortly: __content-based recommendation__ (item-centred) and __collaborative filtering__ (user-centred).

<img width="500px;" height="500px" src="https://www.researchgate.net/profile/Lionel_Ngoupeyou_Tondji/publication/323726564/figure/fig5/AS:631605009846299@1527597777415/Content-based-filtering-vs-Collaborative-filtering-Source.png">

[Source](https://www.researchgate.net/publication/323726564_Web_Recommender_System_for_Job_Seeking_and_Recruiting)

Before we cover the implementation of these, we will cover what is informally referred to as a simple recommender system: a system that uses the weighted average rating from all users to make recommendations on the "best" options. This is also referred to as a type of __demographic recommender system.__ Throughout this notebook, we will use the "Movies Dataset" from [Kaggle](https://www.kaggle.com/rounakbanik/the-movies-dataset), where the full version contains information on over 45,000 movies with 26 million ratings from  270,000 users. We will be using the small version, as shown below:

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
# Importing ratings
ratings = pd.read_csv("../DATA/ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
# Importing movie metadata
metadata = pd.read_csv("../DATA/movies_metadata.csv")
metadata.head()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


For our simple recommender system, we will use the IMDB's known __weighted average formula__ used for their Top Movies Chart, given as follows:

$$ R_{W} = (\frac{v}{v + m})R + (\frac{m}{v + m})C  $$

Where:
- $R_{W}$ is the weighted average movie rating
- $v$ is the number of votes for that movie title
- $m$ is the minimum number of votes required to be in the top Chart
- $R$ is the average rating of that movie title
- $C$ is the mean vote rating across all movies

We can now begin our calculations to construct our simple recommender:

In [5]:
# Computing mean vote count across all movies
vote_counts = metadata[metadata['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = metadata[metadata['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

We must now choose a value for the minimum number. In this case, we will choose a value $m$ that gives us movies that have received more votes than 95% of the other remaining movies.

In [6]:
# Computing minimum number of votes required
m = vote_counts.quantile(0.95)
print(m)

434.0


We can now extract the movies that are considered to be canditates for the top charts in our recommender system given our computed 'm'.

In [7]:
# Extracting all movies that have a votecount that is greater than our m value
qualified = metadata[(metadata['vote_count'] >= m) & (metadata['vote_count'].notnull()) & (metadata['vote_average'].notnull())] \
                 [['title', 'release_date', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [8]:
# Computing weighted average and determining top 250 chart
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['weighted_average'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('weighted_average', ascending=False).head(250)
qualified

,title,release_date,vote_count,vote_average,popularity,genres,weighted_average
15480,Inception,2010-07-14,14075,8,29.1081,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",7.917588
12481,The Dark Knight,2008-07-16,12269,8,123.167,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",7.905871
22879,Interstellar,2014-11-05,11187,8,32.2135,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",7.897107
2843,Fight Club,1999-10-15,9678,8,63.8696,"[{'id': 18, 'name': 'Drama'}]",7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001-12-18,8892,8,32.0707,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",7.871787
...,...,...,...,...,...,...,...
2006,Indiana Jones and the Temple of Doom,1984-05-23,2841,7,15.8023,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",6.767415
16129,The King's Speech,2010-09-06,2817,7,11.2604,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",6.765698
895,Sunset Boulevard,1950-08-10,533,8,11.7098,"[{'id': 18, 'name': 'Drama'}]",6.763480
9888,Sin City,2005-04-01,2755,7,15.0105,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",6.761143


Great, so now we have the top charts. This chart can be carried further to become a simple recommender system by recommending the top movies in the charts to all users. Is this a good recommender system? Not particularly. By taking a global weighted average we are able to determine which ones are considered the best on average, but we are unable to account for the individual preferences of the users. For instance, if I was a fan of exclusively Romcoms, I would be recommended only movies I dislike from the list above. The same would happen to people who really don't like action and thrillers. Therefore, we must use our data to instead be able to account for individual preferences! Therefore, simple recommender systems like this that generate global recommendations are generally only used for users who the system has collected little data from.

By accounting for individual user preferences, we would likely achieve a higher score. But how can we determine which system is better? This leads us to the two methods of recommender system evaluation: __offline evaluation__ and __online evaluation.__

# Offline & Online Evaluation

How can we tell that our recommender system is doing what it is supposed to? There are two different approaches to evaluating our system:

- __Offline evaluation:__ Use data we already have and evaluation metrics to compute numeric efectiveness measures that can be tuned for and/or compared. These are the same evaluation metrics which we have encountered and used to assess the performance of our models
- __Online evaluation:__  involves using a live system, and tracking user-related behaviors such as dwell-times, click-through rates, and purchase conversions

When carrying offline evaluation, we can split our data into a training and a test dataset just as we have seen before to ensure that we are tuning our systems appropriately. On the other hand, online evaluation enables us to capture aspects of the performance of our system that offline methods cannot. Whether offline evaluation, online evaluation or a combination of both is the best method to evaluate our system's performance still remains a topic of research. For the purposes of this notebook, we will only be covering simplistic forms of offline evaluation.

# Content-based Recommenders
We can now begin to understand the first sub-class of recommendation systems: __content-based recommenders.__ Let us look at the recommendation problem in the context of our movies dataset. It is intuitive to say that we would like to recommend romance movies to someone that has rated other romantic movies highly as opposed to action, or to recommend older films to users to who are fans of old classics, or even Batman movies to a Batman fan. In this context, we are looking at the characteristics (content) of each movie, and recommending movies that are similar to the previously highly rated movies by the same user. 

There are multiple approaches for the machinery of content-based recommenders. Most will either use the features of movies to predict whether you like or dislike a movie (classification) or to predict the rating the user would give to a movie they have not yet seen (__model-based__). Some might even use the features of a movie you have just watched and recommend the most similar movies to that given movie given their respective features (__memory-based__). We will be creating our own algorithm to predict the ratings of unseen movies and recommend those that are rated the highest.

In modelling terms, we can frame the problem of recommendation as using the __features__ of movies watched by a user and the ratings given to each movie to __predict__ the rating the user would give to a movie not yet watched based on the movie's features. This is why this approach is referred to as item-centred. In other words, if we have enough data, we train a model for each user based on the previously watched movies and their features. The features we have chosen to use in our model are given genres, vote average, release date and runtime. While release date and vote average are available, we have to extract and process the genres for each movie. Given that we have a list of genres for each movie, we will have to dummy encode it as follows:
- Determine how many genres there are and make each genre a feature of the model
- Assign to each genre a zero if it is not present in the movie's list of genres, or 1 for each genre given each movie 

In [ ]:
# Importing our data
links_small = pd.read_csv("../DATA/links_small.csv")
ratings_small = pd.read_csv("../DATA/ratings_small.csv")
md = pd.read_csv("../DATA/movies_metadata.csv")

In [ ]:
# Displaying our data
# md
# ratings_small
#links_small

In [ ]:
# Only considering movies with the upper 15% most votes
m = md["vote_count"].quantile(0.85)
md = md[md["vote_count"] > m]

In [ ]:
# Making sure imdb_id matches between links and metadata
md["imdb_id"] = md["imdb_id"].str.strip("tt")

# Removing all movies without a genre, release_date, imdb_id or vote_average
md["genres"].replace('[]', np.nan, inplace=True)
md.dropna(subset=["genres", "release_date", "imdb_id", "vote_average"], inplace=True)

# Converting release_date to a POSIX timestamp float
md["release_date"] = pd.to_datetime(md["release_date"], infer_datetime_format=True)
md["release_date"] = md["release_date"].apply(lambda x:x.timestamp())

# Converting imdb_id to int
md["imdb_id"] = md["imdb_id"].astype('int64')

In [ ]:
# Converting "genres" column from a dictionary from a list of strings, containing the respective genres
def extract_genres(x):
    genre_string = ''
    x = eval(x) # executes expression inside of string
    for dictionary in x:
        genre_string += dictionary["name"] + '|'
    return genre_string # include all but last one
md["genres"] = md["genres"].apply(extract_genres)

Now that we have preprocessed the data we are going to use for this model, we can extract the three columns: genre, release_data and budget. We will now also determine the unique features present in the dataset and use each as a feature. Note that this model assumes that all possible genres are included in the dataset.

In [ ]:
# Initialising our features matrix
FEATURES = md[["imdb_id", "original_title", "release_date", "vote_average"]]

# Finding unique genre names
GENRES = md["genres"]
unique_genres = list(set(GENRES.sum().split('|')[:-1])) # Don't include last element
print(unique_genres)

# Removing '|' from the end of each string
GENRES = GENRES.apply(lambda x:x[:-1])

In [ ]:
# Adding each genre as a feature
extended_features = GENRES.str.get_dummies()

# Horizontally stack our extended features and the original features
FEATURES = FEATURES.merge(extended_features, left_index=True, right_index=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Normalizing non-categorical features
scaler = MinMaxScaler()
FEATURES[['release_date', 'vote_average']] = scaler.fit_transform(FEATURES[['release_date', 'vote_average']])

Now that our features for each movie are set, we need to create a list of users, where each user contains the features and ratings of each of the movies they rated. To link the users and their ratings to the movies, we will need to use the intermediary "links" table. Be careful! We have dropped a few of the movies in the original dataset when the required feature was not available. First we will merge the appropriate dataframes.

In [ ]:
# First Join: links JOIN FEATURES ON imdbId
FEATURES = FEATURES.rename(columns={'imdb_id':'imdbId'}) # making column names match
first_join = links_small.merge(FEATURES, on="imdbId")

# Second Join: ratings JOIN first_join on movieId
data_matrix = ratings_small.merge(first_join, on="movieId")

# Delete unnecessary columns
data_matrix.drop(['movieId', 'timestamp', 'imdbId', 'tmdbId'],axis='columns', inplace=True)

After merging our dataframes, we will create a list of users, each with all the ratings provided by each user and features of the corresponding rated movie. But how do we evaluate our errors? A naive approach would be to compute the chosen metric(s) for each user, then average their values over all users. But is that a fair measure? What if one user gave us a test set size of 70 and another a test set 8? Should the results of these be weighted equally?

The answer is no. To account for this imbalance, we would have to remedy this by taking a weighted average given the number of movies rated for a given user. To make things simpler instead of having to compute this weighted average, we will have global trackers of labels and predictions over all users, then apply our metrics over that. This way, our user that contributed to 70 ratings for the test set will have a larger contribution than the user with a test set size of 8 ratings by default! This will become clearer after a closer inspection of the code.

In [ ]:
# Creating unique user list
unique_users = list(set(data_matrix["userId"]))
user_data = {}

# Adding movie ratings and features to the list of the corresponding user for users that have rated more than tr movies
data_copy = data_matrix.copy()
tr = 50 # at least n reviews
for user_id in unique_users:
    current_data = data_copy[data_copy["userId"] == user_id]
    if current_data.shape[0] >= tr:
        user_data[user_id] = current_data
    data_copy = data_copy[data_copy["userId"] != user_id]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_absolute_error as MAE, mean_squared_error as MSE
import random
from tqdm import tqdm

# Train regression model on each user!
user_models = {}
scores = []
labels = []
total_predictions = []
for user_id in tqdm(user_data.keys()):
    # Get user data
    data = user_data[user_id]
    Y = data["rating"]
    X = data.drop(["userId", "rating", "original_title"], axis=1)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
    
    # Train model and predict on test data
    reg = RandomForestRegressor(n_estimators=40, max_depth=3, random_state=0)
    fitted_reg = reg.fit(X_train, Y_train)
    predictions = fitted_reg.predict(X_test)
    user_models[user_id] = fitted_reg # storing estimator for this user!
    labels.extend(Y_test)
    total_predictions.extend(predictions)
    
    # Optional: compute cross validated metrics in your own time (may take some time!)
#     scores.append(cross_validate(reg, X, Y, cv=5, scoring=('neg_mean_absolute_error', 'neg_root_mean_squared_error')))


In [ ]:
# Displaying metrics that measure our models performance
print("MAE:", MAE(labels, total_predictions))
print("RMSE:", MSE(labels, total_predictions, squared=False))

# Displaying metrics for 5-fold cross validation
# print("5-Fold MAE: ", abs(np.mean([score["test_neg_mean_absolute_error"] for score in scores])))
# print("5-fold RMSE: ", abs(np.mean([score["test_neg_root_mean_squared_error"] for score in scores])))

These scores are relatively okay. The MAE is the more easily interpretable metric and tells us that, on average, we deviate from the true rating by about 0.64. The RMSE metric adds more weights to large errors because of how it is calculated, and highlights large errors that may arise by how much larger it is than the RMSE.

Now that we have trained a Random Forest Regressor model on every user, we can now pick any random user and recommend them the movies that receive the highest ratings according to our model.

In [ ]:
import random

user_id = random.choice(list(user_data.keys()))

# Check the predictions of the recommender
model = user_models[user_id]
movies = FEATURES.drop(axis=1, columns=["imdbId"]) # We don't care about the imdbId
# We want the title, but not as our model input
movies["rating"] = model.predict(movies.drop(axis=1, columns=["original_title"])) 

# Diplay top 10 rated and top 10 recommendations!
top_rated = user_data[user_id].nlargest(columns="rating", n=10)
top_rated

In [ ]:
# Top 10 recommendations
movies.nlargest(columns="rating", n=10)

From the evaluation metrics, we can see that the prediction made by our model deviates on average by 0.63 from the true rating, which is not a large deviation, and is generally able to distinguish between 'okay', 'great' and 'terrible' movies in the opinion of the user. However, if we would like to recommend 10 movies out of roughly 45,000, our rating predictions need to be even better. Therefore, this model may serve as a strong baseline model that can be improved by accounting for other features such as cast, directors, plot, amongst others.

### Limitations:
As you have seen, we only considered users that had reviewed 100 movies or more, as we do not have enough data to accurately train a model for that given user for a large number of features. What happens in the real-world if we need to make recommendations for users who have rated few movies? What if they are a completely new user? This is known as the __cold-start__ problem and makes it so that models such as our content-based one struggle to make recommendations for users whom we have little data on, making it an issue of __data sparsity__. Additionally, this approach requires feature extraction, which can sometimes be cumbersome.

# Collaborative-filtering

The second classification of algorithms we are now going to go over is known as __collaborative-filtering__, which, as the name implies, requires the collaboration of all the different members of our user-base. It works under the assumption that users that have had similar preferences/choices in the past have similar tastes. If Susan and Tom both love the same movies, if Susan loves _He's Just Not That Into You_ then we should probably recommend that movie to Tom! Unlike content-based recommender systems, collaborative filtering does not require any manual feature extraction, hence why it's referred to as user-centred.

<img src="https://miro.medium.com/max/676/1*_x4VqIeV9L6fxXm5PeAYsg.png" width="500px" height="500px">

[Source](https://medium.com/sfu-cspmp/recommendation-systems-user-based-collaborative-filtering-using-n-nearest-neighbors-bf7361dc24e0)

As before, there are different approaches to how we can create a collaborative filtering system:
- We can either look at the preferences/ratings of users who have similar interests and predict the preferences/ratings of this user (memory-based)
- We can use the data we have on the preferences/ratings of all users in our dataset to __learn__ and/or __predict__ the features of individual different items and use these features to predict preferences/ratings for given items (model-based)

As we have covered the model-based approach in the content-based recommendation section, we will be implementing a memory-based approach for collaborative filtering by using some of the unsupervised learning techniques we have encountered so far in the course. If you are interested in looking into an implementation that uses feature learning, check out this [video](https://www.youtube.com/watch?v=9AP-DgFBNP4&ab_channel=ArtificialIntelligence-AllinOne).

More specifically, we will be creating a ratings vector for each user and using a KNN with means algorithm to compute mean rating of the user's $K$ nearest neighbours for movies not yet rated by a given user. We will be using the __Surprise library__, a scikit for building and analyzing recommender systems that deal with explicit rating data. Before we use our data we will have to clean it just as in the case for the content-based system. For the Surprise 'cross-validate' method, we need to have our data as three columns in the following order: userId, movieId, rating. 

In [ ]:
import pandas as pd
import numpy as np

# Importing our data
links_small = pd.read_csv("../DATA/links_small.csv")
ratings_small = pd.read_csv("../DATA/ratings_small.csv")
md = pd.read_csv("../DATA/movies_metadata.csv")

In [ ]:
# Filtering movie data by vote count
md = md[md["vote_count"] >= 50]
md.dropna(subset=["imdb_id", "title"], inplace=True)
md.drop_duplicates(subset=["imdb_id"], inplace=True)
md["imdb_id"] = md["imdb_id"].str.strip("tt").astype('int64')

# Merging our data
movies = md[["imdb_id", "title"]]
movies.rename(columns={'imdb_id':'imdbId'}, inplace=True) # making column names match
first_join = links_small.merge(movies, on="imdbId")
joined_data = ratings_small.merge(first_join, on="movieId")
joined_data.drop(['title', 'timestamp', 'tmdbId', 'imdbId'], axis='columns', inplace=True)

In [ ]:
from surprise.prediction_algorithms.knns import  KNNWithMeans
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(0, 5))

data = Dataset.load_from_df(joined_data, reader)
algo = KNNWithMeans()
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Great, so we can see that they carried out the KNNWithMeans and got relatively good scores from the baseline model! But how does it work? Let us look at what the Surprise library is actually doing. What does the input data look like? What does their predictions look like and how is their model evaluated?

In [ ]:
# What does the data look like?
trainset, testset = train_test_split(data, test_size=.25)
# testset

# What do the predictions look like?
# algo = KNNWithMeans().fit(trainset)
# predictions = algo.test(testset)
# predictions

What is this algorithm actually doing? In memory-based approaches with algorithms such as KNN, we generally construct a __user-item matrix,__ as seen in the diagram above. This is a 2D representation of users vs items, such that each row contains all the ratings (and missing ratings) a user gave to every movie. Cells representing movies that have not been rated by the respective users are filled with zeros, NaNs or other placeholders. In the case of this surprise algorithm, they created a __similarity matrix__ containing the measure of cosine similarity between each user pair and uses that similarity as weights when computing the average rating of all its neighbours.

Some of the most popular frameworks for collaborative filtering algorithms use __matrix factorization__ approaches, which include using a variant of SVD (as SVD cannot be applied to a matrix with missing data), known as SVD++. In fact, this is the algorithm popularized by Simon Funk during the Netflix prize for recommender systems! The theory of this algorithm extends beyond the scope of this course, but you can read more about it [here](https://www.hindawi.com/journals/mpe/2017/1975719/). Luckily this algorithm is also implemented by the Surprise library!

[Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD)

In [ ]:
from surprise import SVD

# CF Algorithm using SVD++
algo = SVD()
cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

### Limitations
Just like with the content-based recommender system, a collaborative filtering system also suffers from a cold start problem. In fact, since it is solely based on the ratings/preferences and not on any metadata, it requires much more data. It can also sometimes be difficult to combine and weight the preferences of user neighbors. 

In the real-world, you will rarely see a model that is purely content-based or purely collaborative filtering. Generally, multiple models that capture the predicted preferences differently are combined.

# Hybrid Systems

As the name implies __hybrid systems__ are essentially ensemble models of different recommender systems together. As each approach generally focuses on a different characteristic and suffers from individual limitations, hybrid systems are generally able to give the optimal result. Arguably one of the most successful recommender systems, the one used by _Spotify_, works with this principle. It combines the predictions of the following three key systems:

1. Collaborative filtering to compare an individual's behaviour to other people's taste
2. Natural language processing to analyse the text in each song and find songs of similar lyrics (content-based)
3. Audio modelling of songs' raw audio (content-based)

Unlike other recommendation systems, Spotify uses __implicit feedback__ instead of ratings, which are things such as stream count. This first model gives their system a way to recommend songs based on similar users. The NLP and audio modelling content-based models enable the system to find similarities between items with very few streams which would not otherwise be recommended by the collaborative filtering model due to data sparsity. This is how Spotify is able to recommend even relatively unpopular songs based on a user's personal preferences, which is what made their 'Discover Weekly' feature so popular!

If you would like to practice developing your own recommender systems, a great place to find datasets that can be used for recommender systems is [here](https://github.com/caserec/Datasets-for-Recommender-Systems).

# Congratulations!